# 5. Практические примеры
⭐️ Мы научились определять классы и создавать объекты. Основная их задача — объединять данные и действия над ними. Эти же задачи можно решать обычными функциями и коллекциями (словарями/списками).

Разница в том, что для ряда задач ООП является более подходящим инструментом и позволяет решать их проще. Рассмотрим ещё несколько примеров — это поможет вам развить интуицию и понять, в каких ситуациях стоит рассматривать переход от простых функций к классам.

## ОТСЛЕЖИВАНИЕ СОСТОЯНИЯ

Одно из классических предписаний для классов — у каждого из множества объектов есть некоторые меняющиеся состояния. 

Вернёмся к примеру: есть база клиентов с основной информацией; в реальном времени нам приходит информация о покупках. Запустим промокампанию, чтобы поощрить старых клиентов, которые сделали у нас много заказов, и выдать им скидку:

In [1]:
class Client():
    # Базовые данные
    def __init__(self, email, order_num, registration_year):
        self.email = email
        self.order_num = order_num
        self.registration_year = registration_year
        self.discount = 0

    # Оформление заказа
    def make_order(self, price):
        self.update_discount()
        self.order_num += 1
        # Здесь было бы оформление заказа, но мы просто выведем его цену
        discounted_price = price * (1 - self.discount)
        print(f"Order price for {self.email} is {discounted_price}")

    # Назначение скидки
    def update_discount(self):
        if self.registration_year < 2018 and self.order_num >= 5:
            self.discount = 0.1


# Применение

# Сделаем подобие базы
client_db = [
    Client("max@gmail.com", 2, 2019),
    Client("lova@yandex.ru", 10, 2015),
    Client("german@sberbank.ru", 4, 2017)
]


# Сгенерируем заказы
client_db[0].make_order(100)
# => Order price for max@gmail.com is 100

client_db[1].make_order(200)
# => Order price for lova@yandex.ru is 180.0

client_db[2].make_order(500)
# => Order price for german@sberbank.ru is 500

client_db[2].make_order(500)
# => Order price for german@sberbank.ru is 450.0


Order price for max@gmail.com is 100
Order price for lova@yandex.ru is 180.0
Order price for german@sberbank.ru is 500
Order price for german@sberbank.ru is 450.0


Два важных момента:

- У нас получился простой интерфейс. С функциями нам пришлось бы передавать много параметров или делать вложенный словарь.
- В классах хорошо реализуется скрытая логика и естественное сохранение состояний. В примере на втором и четвёртом заказах автоматически появилась скидка.

### Задание 5.1
Определите класс для пользователей User:

- у него должны быть атрибуты email, password и balance, которые устанавливаются при инициализации;
- у него должен быть метод login, который принимает емайл и пароль. Если они совпадают с атрибутами объекта, он возвращает True, а иначе — False;
- должен быть метод update_balance(amount), который изменяет баланс счёта на величину amount.

In [2]:
class User():
    def __init__(self, email, password, balance):
        self.email = email
        self.password = password
        self.balance = balance

    def login(self, email, password):
        return self.email == email and self.password == password

    def update_balance(self, amount):
        self.balance += amount


В случае правильного описания класса код, приведённый ниже, должен выдать следующий результат:

In [4]:
user = User("gosha@roskino.org", "qwerty", 20_000)
print(user.login("gosha@roskino.org", "qwerty123"))
# => False
print(user.login("gosha@roskino.org", "qwerty"))
# => True
user.update_balance(200)
user.update_balance(-500)
print(user.balance)
# => 19700


False
True
19700


## КОМБИНАЦИЯ ОПЕРАЦИЙ

Классы могут пригодиться, если вы регулярно делаете над данными одну и ту же последовательность разноплановых функций. Вы можете упаковать их в класс и в дальнейшем сразу получать результат по загруженным данным.

У нас есть численные данные из разных источников. Если они в виде строк, то нужно привести их к числам, а пропуски — заполнить значениями. Сделаем доступ к медиане, среднему значению и стандартному отклонению:

In [5]:
import statistics


class DataFrame():
    def __init__(self, column, fill_value=0):
        # Инициализируем атрибуты
        self.column = column
        self.fill_value = fill_value
        # Заполним пропуски
        self.fill_missed()
        # Конвертируем все элементы в числа
        self.to_float()

    def fill_missed(self):
        for i, value in enumerate(self.column):
            if value is None or value == '':
                self.column[i] = self.fill_value

    def to_float(self):
        self.column = [float(value) for value in self.column]

    def median(self):
        return statistics.median(self.column)

    def mean(self):
        return statistics.mean(self.column)

    def deviation(self):
        return statistics.stdev(self.column)


# Воспользуемся классом
df = DataFrame(["1", 17, 4, None, 8])

print(df.column)
# => [1.0, 17.0, 4.0, 0.0, 8.0]
print(df.deviation())
# => 6.89
print(df.median())
# => 4.0


[1.0, 17.0, 4.0, 0.0, 8.0]
6.892024376045111
4.0


Мы получили очень лаконичный интерфейс для использования класса. В __init__ мы использовали значение по умолчанию для fill_value, а методы позволяют нам определять необязательные параметры.

### Задание 5.2
Определите класс IntDataFrame, который принимает список неотрицательных чисел и приводит к целым значениям все числа в этом списке. После этого становится доступен метод count, который считает количество ненулевых элементов, и метод unique, который возвращает число уникальных элементов.

In [8]:
class IntDataFrame():
    def __init__(self, column):
        self.column = [int(value) for value in column]
    
    def count(self):
        return sum(0 if value == 0 else 1 for value in self.column)
    
    def unique(self):
        return len(set(self.column))

В случае правильного описания класса код, приведённый ниже, должен выдать следующий результат:

In [10]:
df = IntDataFrame([4.7, 4, 3, 0, 2.4, 0.3, 4])

print(df.count())
# => 5
print(df.unique())
# => 4


5
4


## КЛАСС-ОБЁРТКА 

Классы можно использовать тогда, когда у вас есть процесс, который требует сложной конфигурации, повторяющейся из раза в раз. Можно написать класс-обёртку, который сведёт этот процесс к одному-двум методам.

Представим, вы делаете обработку данных и в конце каждого дня сохраняете результат в архив. Вы хотите, чтобы данные каждого дня лежали в отдельном файле для этого дня, при этом можно было бы получить данные за произвольный день. 

Перед запуском кода создайте папку с названием archive там же, где находится ноутбук:

In [11]:
import pickle
from datetime import datetime
from os import path


class Dumper():
    def __init__(self, archive_dir="archive/"):
        self.archive_dir = archive_dir

    def dump(self, data):
        # Библиотека pickle позволяет доставать и класть объекты в файл
        with open(self.get_file_name(), 'wb') as file:
            pickle.dump(data, file)

    def load_for_day(self, day):
        file_name = path.join(self.archive_dir, f"{day}.pkl")
        with open(file_name, 'rb') as file:
            sets = pickle.load(file)
        return sets

    # возвращает корректное имя для файла
    def get_file_name(self):
        today = datetime.now().strftime("%y-%m-%d")
        return path.join(self.archive_dir, f"{today}.pkl")


# Пример использования

data = {
    'perfomance': [10, 20, 10],
    'clients': {"Romashka": 10, "Vector": 34}
}


dumper = Dumper()

# Сохраним данные
dumper.dump(data)

# Восстановим для сегодняшней даты
file_name = datetime.now().strftime("%y-%m-%d")
restored_data = dumper.load_for_day(file_name)
print(restored_data)
# => {'perfomance': [10, 20, 10], 'clients': {'Romashka': 10, 'Vector': 34}}


{'perfomance': [10, 20, 10], 'clients': {'Romashka': 10, 'Vector': 34}}


Сохранение и восстановление работает в пару строк. В результате мы можем приводить достаточно сложные операции к простому виду.

### Задание 5.3
Напишите класс сборщика технических сообщений OwnLogger.

- У него должен быть метод log(message, level), который записывает сообщения. Здесь сообщение message может быть любым, а level — один из "info", "warning", "error".
- Также примените метод show_last(level), где level может быть "info", "warning", "error", "all". Для "all" он просто возвращает последнее добавленное сообщение, а для остальных — последнее поступившее сообщение соответствующего уровня.
- При этом по умолчанию значение именно "all". Если подходящего сообщения нет, возвращает None.

In [25]:
class LogMessage():
    def __init__(self, message, level):
        self.message = message
        self.level = level


class OwnLogger():
    def __init__(self):
        self.logbook = []

    def log(self, message, level):
        self.logbook.append(LogMessage(message, level))

    def show_last(self, level='all'):
        if level == 'all':
            return None if len(self.logbook) == 0 else self.logbook[-1].message
        return next((self.logbook[i].message for i in range(len(self.logbook) - 1, -1, -1) if self.logbook[i].level == level), None)


В случае правильного описания класса код, приведённый ниже, должен выдать следующий результат:

In [26]:
logger = OwnLogger()
logger.log("System started", "info")
print(logger.show_last("error"))
# => None
# Некоторые интерпретаторы Python могут не выводить None, тогда в этой проверке у вас будет пустая строка
logger.log("Connection instable", "warning")
logger.log("Connection lost", "error")

print(logger.show_last())
# => Connection lost
print(logger.show_last("info"))
# => System started


None
Connection lost
System started


## ИМПОРТ И ОРГАНИЗАЦИЯ КОДА

Классы, как и библиотечные функции, можно импортировать в другие программы. Для этого нужно положить класс в отдельный файл в корне проекта и использовать ключевое слово import. 

Например, если мы положим Dumper в файл dumper.py в корне проекта, то его можно импортировать командой:

In [29]:
from dumper import Dumper


Пишем from <имя файла без .py> import <имя класса>. Имя файла должно начинаться с буквы и не совпадать с именами библиотечных модулей. Если файлов с классами много, их можно складывать в папки, предварительно положив туда пустой файл __init__.py — это требование Python.

Сгруппируем классы из примеров в папке helpers. Структура файлов:

helpers

-- \__init__.py

-- dumper.py

-- data_frame.py

-- client.py

Импортируем, соединяя все шаги через точку с помощью того же синтаксиса, что использовался для импорта библиотечных функций.

In [ ]:
from helpers.dumper import Dumper
from helpers.data_frame import DataFrame
from helpers.client import Client


## Задание 5.4
Пусть у вас есть папка duck, в которой лежит файл egg.py, а в нём — класс Needle. Какой командой мы можем импортировать этот класс?

In [ ]:
from duck.egg import Needle
